In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset

import torchvision
import torchvision.transforms as transforms

import os
import time
import datetime
from models import ResNet18
from advertorch.attacks import L2PGDAttack

In [2]:
learning_rate = 0.1
file_name = 'basic_training_nonrobust_dataset'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform_train = transforms.Compose([transforms.ToTensor(),])
transform_test = transforms.Compose([transforms.ToTensor(),])

In [3]:
class TensorDataset(Dataset):
    def __init__(self, *tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        im, targ = tuple(tensor[index] for tensor in self.tensors)
        if self.transform:
            real_transform = transforms.Compose([
                transforms.ToPILImage(),
                self.transform
            ])
            im = real_transform(im)
        return im, targ

    def __len__(self):
        return self.tensors[0].size(0)


In [4]:
data_path = "d_non_robust_CIFAR/"
train_data = torch.cat(torch.load(os.path.join(data_path, f"CIFAR_ims")))
train_labels = torch.cat(torch.load(os.path.join(data_path, f"CIFAR_lab")))
train_dataset = TensorDataset(train_data, train_labels, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=0)

Files already downloaded and verified


In [5]:
net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

adversary = L2PGDAttack(net, loss_fn=nn.CrossEntropyLoss(), eps=0.25, nb_iter=100, eps_iter=0.01, rand_init=True, clip_min=0.0, clip_max=1.0, targeted=False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

In [6]:
def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 10 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)

def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    benign_loss = 0
    adv_loss = 0
    benign_correct = 0
    adv_correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        benign_loss += loss.item()

        _, predicted = outputs.max(1)
        benign_correct += predicted.eq(targets).sum().item()

        if batch_idx % 10 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign test loss:', loss.item())

        adv = adversary.perturb(inputs, targets)
        adv_outputs = net(adv)
        loss = criterion(adv_outputs, targets)
        adv_loss += loss.item()

        _, predicted = adv_outputs.max(1)
        adv_correct += predicted.eq(targets).sum().item()

        if batch_idx % 10 == 0:
            print('Current adversarial test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current adversarial test loss:', loss.item())

    print('\nTotal benign test accuarcy:', 100. * benign_correct / total)
    print('Total adversarial test Accuarcy:', 100. * adv_correct / total)
    print('Total benign test loss:', benign_loss)
    print('Total adversarial test loss:', adv_loss)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [7]:
t1 = time.time()
for epoch in range(0, 200):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    #test(epoch)
t2 = time.time()


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1015625
Current benign train loss: 2.390305995941162

Current batch: 10
Current benign train accuracy: 0.1328125
Current benign train loss: 3.301467180252075

Current batch: 20
Current benign train accuracy: 0.109375
Current benign train loss: 2.869253158569336

Current batch: 30
Current benign train accuracy: 0.09375
Current benign train loss: 2.555495500564575

Current batch: 40
Current benign train accuracy: 0.109375
Current benign train loss: 2.4002902507781982

Current batch: 50
Current benign train accuracy: 0.109375
Current benign train loss: 2.2988736629486084

Current batch: 60
Current benign train accuracy: 0.1015625
Current benign train loss: 2.3203155994415283

Current batch: 70
Current benign train accuracy: 0.125
Current benign train loss: 2.3195948600769043

Current batch: 80
Current benign train accuracy: 0.125
Current benign train loss: 2.2987494468688965

Current batch: 90
Current benign train accu


Current batch: 370
Current benign train accuracy: 0.078125
Current benign train loss: 2.3038485050201416

Current batch: 380
Current benign train accuracy: 0.1328125
Current benign train loss: 2.2993903160095215

Current batch: 390
Current benign train accuracy: 0.075
Current benign train loss: 2.3240365982055664

Total benign train accuarcy: 9.768
Total benign train loss: 901.1896877288818

[ Train epoch: 2 ]

Current batch: 0
Current benign train accuracy: 0.109375
Current benign train loss: 2.2999391555786133

Current batch: 10
Current benign train accuracy: 0.0703125
Current benign train loss: 2.309260845184326

Current batch: 20
Current benign train accuracy: 0.0703125
Current benign train loss: 2.311772346496582

Current batch: 30
Current benign train accuracy: 0.1171875
Current benign train loss: 2.3035993576049805

Current batch: 40
Current benign train accuracy: 0.0859375
Current benign train loss: 2.303835391998291

Current batch: 50
Current benign train accuracy: 0.109375
C


Current batch: 330
Current benign train accuracy: 0.15625
Current benign train loss: 2.2864067554473877

Current batch: 340
Current benign train accuracy: 0.03125
Current benign train loss: 2.3070151805877686

Current batch: 350
Current benign train accuracy: 0.09375
Current benign train loss: 2.306865692138672

Current batch: 360
Current benign train accuracy: 0.1015625
Current benign train loss: 2.2996582984924316

Current batch: 370
Current benign train accuracy: 0.0859375
Current benign train loss: 2.3019051551818848

Current batch: 380
Current benign train accuracy: 0.09375
Current benign train loss: 2.307360887527466

Current batch: 390
Current benign train accuracy: 0.0375
Current benign train loss: 2.302995443344116

Total benign train accuarcy: 10.062
Total benign train loss: 900.8780307769775

[ Train epoch: 4 ]

Current batch: 0
Current benign train accuracy: 0.1015625
Current benign train loss: 2.299938678741455

Current batch: 10
Current benign train accuracy: 0.109375
Cu


Current batch: 300
Current benign train accuracy: 0.2265625
Current benign train loss: 2.036699056625366

Current batch: 310
Current benign train accuracy: 0.1875
Current benign train loss: 2.016622304916382

Current batch: 320
Current benign train accuracy: 0.1484375
Current benign train loss: 2.1619436740875244

Current batch: 330
Current benign train accuracy: 0.1875
Current benign train loss: 2.125295639038086

Current batch: 340
Current benign train accuracy: 0.2421875
Current benign train loss: 1.9967334270477295

Current batch: 350
Current benign train accuracy: 0.28125
Current benign train loss: 2.0083656311035156

Current batch: 360
Current benign train accuracy: 0.25
Current benign train loss: 1.98556387424469

Current batch: 370
Current benign train accuracy: 0.28125
Current benign train loss: 2.0567984580993652

Current batch: 380
Current benign train accuracy: 0.171875
Current benign train loss: 2.0442538261413574

Current batch: 390
Current benign train accuracy: 0.175
C


Current batch: 260
Current benign train accuracy: 0.3203125
Current benign train loss: 1.748002290725708

Current batch: 270
Current benign train accuracy: 0.34375
Current benign train loss: 1.856491208076477

Current batch: 280
Current benign train accuracy: 0.375
Current benign train loss: 1.7384450435638428

Current batch: 290
Current benign train accuracy: 0.3046875
Current benign train loss: 1.9271347522735596

Current batch: 300
Current benign train accuracy: 0.3046875
Current benign train loss: 1.9307085275650024

Current batch: 310
Current benign train accuracy: 0.3671875
Current benign train loss: 1.7323647737503052

Current batch: 320
Current benign train accuracy: 0.34375
Current benign train loss: 1.8032420873641968

Current batch: 330
Current benign train accuracy: 0.3203125
Current benign train loss: 1.8654823303222656

Current batch: 340
Current benign train accuracy: 0.3515625
Current benign train loss: 1.8337661027908325

Current batch: 350
Current benign train accura

...


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0011431531747803092

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0009192854631692171

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0005806695553474128

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0010306218173354864

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0007711116340942681

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0009262770181521773

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0009616322931833565

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0006121335318312049

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0008012566831894219

Current batch: 100
Current benign train accuracy: 1.0
Current benign tra


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0010100340005010366

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0005753173609264195

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0006744254496879876

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0007516302284784615

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0006506580393761396

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0008835592889226973

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0005766982794739306

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0005347500555217266

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0007079302449710667

Current batch: 100
Current benign train accuracy: 1.0
Current benign tra


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0008798362105153501

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0005839897785335779

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.0006644126260653138

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0008115948876366019

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.000542761932592839

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0007951739244163036

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0008581706206314266

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.000557052728254348

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0008109943009912968

Current batch: 100
Current benign train accuracy: 1.0
Current benign train


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0009324643760919571

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.000592379248701036

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.000807015341706574

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0008039092645049095

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0010442573111504316

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0006064261542633176

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0007144810515455902

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.000671686721034348

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0006895535625517368

Current batch: 100
Current benign train accuracy: 1.0
Current benign train 


Current batch: 10
Current benign train accuracy: 1.0
Current benign train loss: 0.0005310274427756667

Current batch: 20
Current benign train accuracy: 1.0
Current benign train loss: 0.0008801943040452898

Current batch: 30
Current benign train accuracy: 1.0
Current benign train loss: 0.000676698749884963

Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.0008105753222480416

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.0005889528547413647

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0007315828115679324

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0013367782812565565

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0006697265198454261

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0006347130401991308

Current batch: 100
Current benign train accuracy: 1.0
Current benign trai

In [9]:
deltatime = str(datetime.timedelta(seconds=t2-t1))
print('Total running time is %s '% deltatime)

Total running time is 1:25:55.955488 


In [10]:
test(epoch)


[ Test epoch: 199 ]

Current batch: 0
Current benign test accuracy: 0.74
Current benign test loss: 0.8781664371490479
Current adversarial test accuracy: 0.01
Current adversarial test loss: 15.753746032714844

Current batch: 10
Current benign test accuracy: 0.84
Current benign test loss: 0.7170594930648804
Current adversarial test accuracy: 0.0
Current adversarial test loss: 15.856802940368652

Current batch: 20
Current benign test accuracy: 0.84
Current benign test loss: 0.558463454246521
Current adversarial test accuracy: 0.01
Current adversarial test loss: 15.598440170288086

Current batch: 30
Current benign test accuracy: 0.83
Current benign test loss: 0.6884454488754272
Current adversarial test accuracy: 0.01
Current adversarial test loss: 15.248276710510254

Current batch: 40
Current benign test accuracy: 0.78
Current benign test loss: 0.7769624590873718
Current adversarial test accuracy: 0.0
Current adversarial test loss: 16.2155818939209

Current batch: 50
Current benign test a